In [25]:
import torch
import torch.nn as nn
from DNN_tools import *
import random
from torch.utils.data.dataloader import DataLoader
import numpy as np
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import mean_squared_error
from math import sqrt
import copy
from physical_dc import *


def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


# 设置随机数种子
setup_seed(27)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

定义参数

In [26]:
train_length  = 0.8
batch_size    = 100
num_epochs    = 200
step_size     = 40 # 学习率变化周期
gamma         = 0.2 # 学习率更新值
learning_rate = 0.001 # 学习率

# 模型输出迭代初始值
S0            = 500
k_p0          = 0.0001
k_n0          = 0.00002
theta_e0      = 1000

数据预处理

In [27]:
# 创建 dataset
dataset_1 = create_dataset(1)
dataset_2 = create_dataset(2)
dataset_3 = create_dataset(3)
dataset_4 = create_dataset(4)

# 将数据的前 train_length 的数据作为训练集 train_set
train_size_1 = int(dataset_1.shape[0] * train_length)
train_size_2 = int(dataset_2.shape[0] * train_length)
train_size_3 = int(dataset_3.shape[0] * train_length)
train_size_4 = int(dataset_4.shape[0] * train_length)
test_size = []
test_size.append(dataset_1.shape[0] * (1 - train_length))
test_size.append(dataset_2.shape[0] * (1 - train_length))
test_size.append(dataset_3.shape[0] * (1 - train_length))
test_size.append(dataset_4.shape[0] * (1 - train_length))

# 分离出训练集和测试集
train_data = []
test_data = []
train_data.append(dataset_1[0:train_size_1])
test_data.append(dataset_1[train_size_1:])
train_data.append(dataset_2[0:train_size_2])
test_data.append(dataset_2[train_size_2:])
train_data.append(dataset_3[0:train_size_3])
test_data.append(dataset_3[train_size_3:])
train_data.append(dataset_4[0:train_size_4])
test_data.append(dataset_4[train_size_4:])

train = np.concatenate((train_data[0], train_data[1], train_data[2], train_data[3]), axis=0)
np.random.shuffle(train)


# 数据归一化
scaler, train_scaled = train_scale(train)
dataset = DataPrepare(train_scaled) # 设置 inputs 和 labels
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, drop_last=True)

PCNN for VRFB Identification

In [28]:
# 定义全连接神经网络类

class DNN(nn.Module):
    def __init__(self, input_dim=5, output_dim=4, hidden_layers=[256, 128, 64, 32, 16, 8]):
        super(DNN, self).__init__()
        
        # 定义隐藏层
        self.hidden_layers = nn.ModuleList()
        layer_sizes = [input_dim] + hidden_layers
        for i in range(len(hidden_layers)):
            self.hidden_layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            self.hidden_layers.append(nn.ReLU())  # 使用ReLU作为激活函数

        # 定义输出层
        self.output1_layer = nn.Linear(hidden_layers[-1], output_dim)
        self.sigmod = nn.Sigmoid()
        # self.output2_layer = nn.Linear(output_dim, 1)

    def forward(self, x):
        for linear in self.hidden_layers:
            x = linear(x)
        out1 = self.output1_layer(self.sigmod(x))
        # out2 = self.output2_layer(out1)
        # return out1, out2
        return out1


# def physical_constraints(inputs, preds, labels):
#     S = preds[:, 0]
#     k_p = preds[:, 1]
#     k_n = preds[:, 2]
#     theta_e = preds[:, 3]
#     S = S0 * torch.exp(S).cpu()
#     k_p = k_p0 * torch.exp(k_p).cpu()
#     k_n = k_n0 * torch.exp(k_n).cpu()
#     theta_e = theta_e0 * torch.exp(theta_e).cpu()
#     T, I, SoC, Q, C_V0, U = train_invert_scale(scaler, inputs.view(-1, 5).cpu().numpy(),
#                                                labels.view(-1, 1).cpu().detach().numpy())
#     C_2, C_3, C_4, C_5, C_Hn, C_Hp, C_H2Op = get_con(SoC, C_V0, C_Hp0, C_Hn0, C_H2Op0)
#     e_con = E_con(T, I, Q, C_2, C_3, C_4, C_5)
#     e_act = E_act(T, I, S, k_p, k_n, C_2, C_3, C_4, C_5)
#     e_ohm = E_ohm(theta_e, T, I)
#     e_ocv = E_ocv(T, C_2, C_3, C_4, C_5, C_Hp, C_Hn, C_H2Op)
#     e_cell = e_con + e_act + e_ohm + e_ocv
#     return e_cell, U


# 创建模型实例
model = DNN().to(device)

# 定义损失函数和优化器
loss_function = nn.MSELoss()
loss_function = loss_function.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # Adam 优化器
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

DNN 训练

In [29]:
# 训练模型
min_epochs = 10
best_model = None
min_loss = 0.2

for epoch in range(num_epochs):
    train_loss = []
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.reshape(batch_size, 5)


        # 梯度清零
        optimizer.zero_grad()
        inputs = inputs.to(torch.float32)
        labels = labels.to(torch.float32)

        # 前向传播
        # preds1, preds2 = model(inputs)
        preds = model(inputs)

        # 计算损失
        # e_cell, U = physical_constraints(inputs, preds, labels)
        S = preds[:, 0]
        k_p = preds[:, 1]
        k_n = preds[:, 2]
        theta_e = preds[:, 3]
        S = S0 * torch.exp(S).cpu()
        k_p = k_p0 * torch.exp(k_p).cpu()
        k_n = k_n0 * torch.exp(k_n).cpu()
        theta_e = theta_e0 * torch.exp(theta_e).cpu()
        T, I, SoC, Q, C_V0, U = train_invert_scale(scaler, inputs.view(-1, 5).cpu().numpy(),
                                                labels.view(-1, 1).cpu().detach().numpy())
        C_2, C_3, C_4, C_5, C_Hn, C_Hp, C_H2Op = get_con(SoC, C_V0, C_Hp0, C_Hn0, C_H2Op0)
        e_con = E_con(T, I, Q, C_2, C_3, C_4, C_5)
        e_act = E_act(T, I, S, k_p, k_n, C_2, C_3, C_4, C_5)
        e_ohm = E_ohm(theta_e, T, I)
        e_ocv = E_ocv(T, C_2, C_3, C_4, C_5, C_Hp, C_Hn, C_H2Op)
        e_cell = e_con + e_act + e_ohm + e_ocv
        
        loss = loss_function(e_cell, U)
        train_loss.append(loss.cpu().item())

        # 更新梯度
        loss.backward()

        # 优化参数
        optimizer.step()  # 更新每个网络的权重

    scheduler.step()  # 调整学习率

    if epoch > min_epochs and loss < min_loss:
        min_val_loss = loss
        best_model = copy.deepcopy(model)

    print('epoch {} train_loss {:.8f}'.format(epoch, np.mean(train_loss)))

    S = S.mean().item()
    k_p = k_p.mean().item()
    k_n = k_n.mean().item()
    theta_e = theta_e.mean().item()
    print('epoch {} S {:.8f} k_p {:.8f} k_n {:.8f} theta_e {:.8f}'.format(epoch, S, k_p, k_n, theta_e))
    
    model.train()

    torch.save(model, f'./result/DNN_model.pth')

epoch 0 train_loss 0.01499264
epoch 0 S 86.59621429 k_p 0.00001494 k_n 0.00000520 theta_e 507.41839600
epoch 1 train_loss 0.00460094
epoch 1 S 87.38838196 k_p 0.00001342 k_n 0.00000567 theta_e 271.91534424
epoch 2 train_loss 0.00424167
epoch 2 S 98.80868530 k_p 0.00001357 k_n 0.00000689 theta_e 101.97662354
epoch 3 train_loss 0.00374706
epoch 3 S 117.54332733 k_p 0.00001452 k_n 0.00000876 theta_e 59.42741776
epoch 4 train_loss 0.00357555
epoch 4 S 127.19138336 k_p 0.00001421 k_n 0.00001012 theta_e 51.26027298
epoch 5 train_loss 0.00354289
epoch 5 S 130.84219360 k_p 0.00001352 k_n 0.00001097 theta_e 49.12707520
epoch 6 train_loss 0.00353416
epoch 6 S 132.28623962 k_p 0.00001296 k_n 0.00001150 theta_e 48.50928879
epoch 7 train_loss 0.00353134
epoch 7 S 132.96691895 k_p 0.00001260 k_n 0.00001183 theta_e 48.31154633
epoch 8 train_loss 0.00353050
epoch 8 S 133.34535217 k_p 0.00001241 k_n 0.00001202 theta_e 48.24108505
epoch 9 train_loss 0.00353032
epoch 9 S 133.58215332 k_p 0.00001231 k_n 0